# 8章
- 以下で実行するコードには確率的な処理が含まれていることがあり、コードの出力結果と本書に記載されている出力例が異なることがあります。

In [1]:
# 8-1
!mkdir chap8
%cd ./chap8

mkdir: cannot create directory ‘chap8’: File exists
/home/root/work/chap8


In [2]:
# 8-2
!pip install transformers==4.18.0 fugashi==1.1.0 ipadic==1.0.0 pytorch-lightning==1.6.1

In [3]:
# 8-3
import itertools
import random
import json
from tqdm import tqdm
import numpy as np
import unicodedata

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForTokenClassification
import pytorch_lightning as pl

# 日本語学習済みモデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

In [4]:
# 8-4
normalize = lambda s: unicodedata.normalize("NFKC",s)
print(f'ＡＢＣ -> {normalize("ＡＢＣ")}' )  # 全角アルファベット
print(f'ABC -> {normalize("ABC")}' )        # 半角アルファベット
print(f'１２３ -> {normalize("１２３")}' )  # 全角数字
print(f'123 -> {normalize("123")}' )        # 半角数字
print(f'アイウ -> {normalize("アイウ")}' )  # 全角カタカナ
print(f'ｱｲｳ -> {normalize("ｱｲｳ")}' )        # 半角カタカナ

ＡＢＣ -> ABC
ABC -> ABC
１２３ -> 123
123 -> 123
アイウ -> アイウ
ｱｲｳ -> アイウ


In [5]:
# 8-5
class NER_tokenizer(BertJapaneseTokenizer):
       
    def encode_plus_tagged(self, text, entities, max_length):
        """
        文章とそれに含まれる固有表現が与えられた時に、
        符号化とラベル列の作成を行う。
        """
        # 固有表現の前後でtextを分割し、それぞれのラベルをつけておく。
        entities = sorted(entities, key=lambda x: x['span'][0])
        splitted = [] # 分割後の文字列を追加していく
        position = 0
        for entity in entities:
            start = entity['span'][0]
            end = entity['span'][1]
            label = entity['type_id']
            # 固有表現ではないものには0のラベルを付与
            splitted.append({'text':text[position:start], 'label':0}) 
            # 固有表現には、固有表現のタイプに対応するIDをラベルとして付与
            splitted.append({'text':text[start:end], 'label':label}) 
            position = end
        splitted.append({'text': text[position:], 'label':0})
        splitted = [ s for s in splitted if s['text'] ] # 長さ0の文字列は除く

        # 分割されたそれぞれの文字列をトークン化し、ラベルをつける。
        tokens = [] # トークンを追加していく
        labels = [] # トークンのラベルを追加していく
        for text_splitted in splitted:
            text = text_splitted['text']
            label = text_splitted['label']
            tokens_splitted = self.tokenize(text)
            labels_splitted = [label] * len(tokens_splitted)
            tokens.extend(tokens_splitted)
            labels.extend(labels_splitted)

        # 符号化を行いBERTに入力できる形式にする。
        input_ids = self.convert_tokens_to_ids(tokens)
        encoding = self.prepare_for_model(
            input_ids, 
            max_length=max_length, 
            padding='max_length', 
            truncation=True
        ) # input_idsをencodingに変換
        # 特殊トークン[CLS]、[SEP]のラベルを0にする。
        labels = [0] + labels[:max_length-2] + [0] 
        # 特殊トークン[PAD]のラベルを0にする。
        labels = labels + [0]*( max_length - len(labels) ) 
        encoding['labels'] = labels

        return encoding

    def encode_plus_untagged(
        self, text, max_length=None, return_tensors=None
    ):
        """
        文章をトークン化し、それぞれのトークンの文章中の位置も特定しておく。
        """
        # 文章のトークン化を行い、
        # それぞれのトークンと文章中の文字列を対応づける。
        tokens = [] # トークンを追加していく。
        tokens_original = [] # トークンに対応する文章中の文字列を追加していく。
        words = self.word_tokenizer.tokenize(text) # MeCabで単語に分割
        for word in words:
            # 単語をサブワードに分割
            tokens_word = self.subword_tokenizer.tokenize(word) 
            tokens.extend(tokens_word)
            if tokens_word[0] == '[UNK]': # 未知語への対応
                tokens_original.append(word)
            else:
                tokens_original.extend([
                    token.replace('##','') for token in tokens_word
                ])

        # 各トークンの文章中での位置を調べる。（空白の位置を考慮する）
        position = 0
        spans = [] # トークンの位置を追加していく。
        for token in tokens_original:
            l = len(token)
            while 1:
                if token != text[position:position+l]:
                    position += 1
                else:
                    spans.append([position, position+l])
                    position += l
                    break

        # 符号化を行いBERTに入力できる形式にする。
        input_ids = self.convert_tokens_to_ids(tokens) 
        encoding = self.prepare_for_model(
            input_ids, 
            max_length=max_length, 
            padding='max_length' if max_length else False, 
            truncation=True if max_length else False
        )
        sequence_length = len(encoding['input_ids'])
        # 特殊トークン[CLS]に対するダミーのspanを追加。
        spans = [[-1, -1]] + spans[:sequence_length-2] 
        # 特殊トークン[SEP]、[PAD]に対するダミーのspanを追加。
        spans = spans + [[-1, -1]] * ( sequence_length - len(spans) ) 

        # 必要に応じてtorch.Tensorにする。
        if return_tensors == 'pt':
            encoding = { k: torch.tensor([v]) for k, v in encoding.items() }

        return encoding, spans

    def convert_bert_output_to_entities(self, text, labels, spans):
        """
        文章、ラベル列の予測値、各トークンの位置から固有表現を得る。
        """
        # labels, spansから特殊トークンに対応する部分を取り除く
        labels = [label for label, span in zip(labels, spans) if span[0] != -1]
        spans = [span for span in spans if span[0] != -1]

        # 同じラベルが連続するトークンをまとめて、固有表現を抽出する。
        entities = []
        for label, group \
            in itertools.groupby(enumerate(labels), key=lambda x: x[1]):
            
            group = list(group)
            start = spans[group[0][0]][0]
            end = spans[group[-1][0]][1]

            if label != 0: # ラベルが0以外ならば、新たな固有表現として追加。
                entity = {
                    "name": text[start:end],
                    "span": [start, end],
                    "type_id": label
                }
                entities.append(entity)

        return entities

In [6]:
# 8-6
tokenizer = NER_tokenizer.from_pretrained(MODEL_NAME)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'NER_tokenizer'.


In [7]:
# 8-7
text = '昨日のみらい事務所との打ち合わせは順調だった。'
entities = [
    {'name': 'みらい事務所', 'span': [3,9], 'type_id': 1}
]

encoding = tokenizer.encode_plus_tagged(
    text, entities, max_length=20
)
print(encoding)

{'input_ids': [2, 10271, 28486, 5, 546, 10780, 2464, 13, 5, 1878, 2682, 9, 10750, 308, 10, 8, 3, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0], 'labels': [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [8]:
# 8-8
text = '騰訊の英語名はTencent Holdings Ltdである。'
encoding, spans = tokenizer.encode_plus_untagged(
    text, return_tensors='pt'
)
print('# encoding')
print(encoding)
print('# spans')
print(spans)

# encoding
{'input_ids': tensor([[    2,     1, 26280,     5,  1543,   125,     9,  6749, 28550,  2953,
         28550, 28566, 21202, 28683, 14050, 12475,    12,    31,     8,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
# spans
[[-1, -1], [0, 1], [1, 2], [2, 3], [3, 5], [5, 6], [6, 7], [7, 9], [9, 10], [10, 12], [12, 13], [13, 14], [15, 18], [18, 19], [19, 23], [24, 27], [27, 28], [28, 30], [30, 31], [-1, -1]]


In [9]:
# 8-9
labels_predicted = [0,1,1,0,0,0,0,1,1,1,1,1,1,1,1,1,0,0,0,0]
entities = tokenizer.convert_bert_output_to_entities(
    text, labels_predicted, spans
)
print(entities)

[{'name': '騰訊', 'span': [0, 2], 'type_id': 1}, {'name': 'Tencent Holdings Ltd', 'span': [7, 27], 'type_id': 1}]


In [10]:
# 8-10
tokenizer = NER_tokenizer.from_pretrained(MODEL_NAME)
bert_tc = BertForTokenClassification.from_pretrained(
    MODEL_NAME, num_labels=4
)
bert_tc = bert_tc.cuda()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'NER_tokenizer'.
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expecte

In [11]:
# 8-11
text = 'AさんはB大学に入学した。'

# 符号化を行い、各トークンの文章中での位置も特定しておく。
encoding, spans = tokenizer.encode_plus_untagged(
    text, return_tensors='pt'
) 
encoding = { k: v.cuda() for k, v in encoding.items() } 

# BERTでトークン毎の分類スコアを出力し、スコアの最も高いラベルを予測値とする。
with torch.no_grad():
    output = bert_tc(**encoding)
    scores = output.logits
    labels_predicted = scores[0].argmax(-1).cpu().numpy().tolist()

# ラベル列を固有表現に変換
entities = tokenizer.convert_bert_output_to_entities(
    text, labels_predicted, spans
)
print(entities)

[{'name': 'AさんはB', 'span': [0, 5], 'type_id': 1}, {'name': '大学', 'span': [5, 7], 'type_id': 3}, {'name': 'に', 'span': [7, 8], 'type_id': 1}, {'name': '入学', 'span': [8, 10], 'type_id': 3}, {'name': 'し', 'span': [10, 11], 'type_id': 1}, {'name': 'た', 'span': [11, 12], 'type_id': 3}, {'name': '。', 'span': [12, 13], 'type_id': 1}]


In [24]:
# 8-12
data = [
    {
        'text': 'AさんはB大学に入学した。',
        'entities': [
            {'name': 'A', 'span': [0, 1], 'type_id': 2},
            {'name': 'B大学', 'span': [4, 7], 'type_id': 1}
        ]
    },
    {
        'text': 'CDE株式会社は新製品「E」を販売する。',
        'entities': [
            {'name': 'CDE株式会社', 'span': [0, 7], 'type_id': 1},
            {'name': 'E', 'span': [12, 13], 'type_id': 3}
        ]
    }
]

# 各データを符号化し、データローダを作成する。
max_length=32
dataset_for_loader = []
for sample in data:
    text = sample['text']
    entities = sample['entities']
    encoding = tokenizer.encode_plus_tagged(
        text, entities, max_length=max_length
    )
    encoding = { k: torch.tensor(v) for k, v in encoding.items() }
    dataset_for_loader.append(encoding)
dataloader = DataLoader(dataset_for_loader, batch_size=len(data))

# ミニバッチを取り出し損失を得る。
for batch in dataloader:
    batch = { k: v.cuda() for k, v in batch.items() } # GPU
    output = bert_tc(**batch) # BERTへ入力
    loss = output.loss # 損失

loss

tensor(0.7445, device='cuda:0', grad_fn=<NllLossBackward0>)

In [13]:
# 8-13
!git clone --branch v2.0 https://github.com/stockmarkteam/ner-wikipedia-dataset 

fatal: destination path 'ner-wikipedia-dataset' already exists and is not an empty directory.


In [14]:
# 8-14
# データのロード
dataset = json.load(open('ner-wikipedia-dataset/ner.json','r'))

# 固有表現のタイプとIDを対応付る辞書 
type_id_dict = {
    "人名": 1,
    "法人名": 2,
    "政治的組織名": 3,
    "その他の組織名": 4,
    "地名": 5,
    "施設名": 6,
    "製品名": 7,
    "イベント名": 8
}

# カテゴリーをラベルに変更、文字列の正規化する。
for sample in dataset:
    sample['text'] = unicodedata.normalize('NFKC', sample['text'])
    for e in sample["entities"]:
        e['type_id'] = type_id_dict[e['type']]
        del e['type']

# データセットの分割
random.shuffle(dataset)
n = len(dataset)
n_train = int(n*0.6)
n_val = int(n*0.2)
dataset_train = dataset[:n_train]
dataset_val = dataset[n_train:n_train+n_val]
dataset_test = dataset[n_train+n_val:]

In [15]:
# 8-15
def create_dataset(tokenizer, dataset, max_length):
    """
    データセットをデータローダに入力できる形に整形。
    """
    dataset_for_loader = []
    for sample in dataset:
        text = sample['text']
        entities = sample['entities']
        encoding = tokenizer.encode_plus_tagged(
            text, entities, max_length=max_length
        )
        encoding = { k: torch.tensor(v) for k, v in encoding.items() }
        dataset_for_loader.append(encoding)
    return dataset_for_loader

# トークナイザのロード
tokenizer = NER_tokenizer.from_pretrained(MODEL_NAME)

# データセットの作成
max_length = 128
dataset_train_for_loader = create_dataset(
    tokenizer, dataset_train, max_length
)
dataset_val_for_loader = create_dataset(
    tokenizer, dataset_val, max_length
)

# データローダの作成
dataloader_train = DataLoader(
    dataset_train_for_loader, batch_size=32, shuffle=True
)
dataloader_val = DataLoader(dataset_val_for_loader, batch_size=256)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'NER_tokenizer'.


In [16]:
# 8-16
# PyTorch Lightningのモデル
class BertForTokenClassification_pl(pl.LightningModule):
        
    def __init__(self, model_name, num_labels, lr):
        super().__init__()
        self.save_hyperparameters()
        self.bert_tc = BertForTokenClassification.from_pretrained(
            model_name,
            num_labels=num_labels
        )
        
    def training_step(self, batch, batch_idx):
        output = self.bert_tc(**batch)
        loss = output.loss
        self.log('train_loss', loss)
        return loss
        
    def validation_step(self, batch, batch_idx):
        output = self.bert_tc(**batch)
        val_loss = output.loss
        self.log('val_loss', val_loss)
        
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='model/'
)

trainer = pl.Trainer(
    gpus=1,
    max_epochs=5,
    callbacks=[checkpoint]
)

# ファインチューニング
model = BertForTokenClassification_pl(
    MODEL_NAME, num_labels=9, lr=1e-5
)
trainer.fit(model, dataloader_train, dataloader_val)
best_model_path = checkpoint.best_model_path

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSeq

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [25]:
# 8-17
def predict(text, tokenizer, bert_tc):
    """
    BERTで固有表現抽出を行うための関数。
    """
    # 符号化
    encoding, spans = tokenizer.encode_plus_untagged(
        text, return_tensors='pt'
    )
    encoding = { k: v.cuda() for k, v in encoding.items() }

    # ラベルの予測値の計算
    with torch.no_grad():
        output = bert_tc(**encoding)
        scores = output.logits
        labels_predicted = scores[0].argmax(-1).cpu().numpy().tolist() 

    # ラベル列を固有表現に変換
    entities = tokenizer.convert_bert_output_to_entities(
        text, labels_predicted, spans
    )

    return entities

# トークナイザのロード
tokenizer = NER_tokenizer.from_pretrained(MODEL_NAME)

# ファインチューニングしたモデルをロードし、GPUにのせる。
model = BertForTokenClassification_pl.load_from_checkpoint(
    best_model_path
)
bert_tc = model.bert_tc.cuda()

# 固有表現抽出
# 注：以下ではコードのわかりやすさのために、1データづつ処理しているが、
# バッチ化して処理を行った方が処理時間は短い
entities_list = [] # 正解の固有表現を追加していく。
entities_predicted_list = [] # 抽出された固有表現を追加していく。
for sample in tqdm(dataset_test):
    text = sample['text']
    print(text)
    entities_predicted = predict(text, tokenizer, bert_tc) # BERTで予測
    entities_list.append(sample['entities'])
    entities_predicted_list.append( entities_predicted )

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'NER_tokenizer'.
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expecte

2019年、世界中の全ての国または地域に1つの系外惑星系を命名する機会を提供する「IAU100 Name ExoWorldsプロジェクト」において、HAT-P-14とHAT-P-14bはオーストリア共和国に割り当てられる系外惑星系となった。
1648年、ロシア人セミヨン・デジニョフが本海峡に到達したがアメリカ大陸を隔てる海峡とは認識しなかったため、1728年に海峡を通過し二つの大陸を隔てる海峡であることを発見したロシア人ヴィトゥス・ベーリングの名が付けられている。
その後1955年の選挙で道議に返り咲き、1957年1月9日に労農党道本部と社会党道連が合同したのに伴い社会党に入党した。
バスケットボール部に入部し、全日本実業団選手権連覇に貢献。
俳優以外では、ルイ・ド・フュネスは常連の監督や脚本家と共同作業をした。
株式会社ナムコに入社し、ファミスタ'88等の作曲を手掛ける。
また、2015年3月のホームページリニューアル以降、バスラインナップで車両紹介がなされている。
大熊良樹は、日本の社会教育家、生命尊重活動家、武道家、絵本作家。
2009年1月にはアメリカ合衆国のポートランド・ティンバーズに加入。
新聞記者であったこともあり、「堀内浩庵」というペンネームで執筆活動を行っている。
reparto corse製造または組み立てのモデルには「Hand made in Italy」というデカールまたはステッカーが貼られている。
フェラーリは2017年8月23日に、同年9月に開催される予定のフランクフルトモーターショーにて、カリフォルニアTの後継車にあたる、ポルトフィーノをワールドプレミアすると発表した。
蕎麦屋からの転業が多かったため、スープは、鰹節や煮干しといった魚介系スープが基本の和風で、濃口醤油を使用した濃い色のスープが特徴である。
正応5年に68歳で崩じ、粟田山陵に葬られた他、聖徳太子の墓所がある叡福寺内に分骨されて宝篋印塔が建てられた。
2020年のオーストラリア国立会計監査局によると、2016年に試算された500億豪ドルを現在の為替レート換算した場合、897億豪ドルになると指摘した。
補償に充てていた1/4が新規債券の割増金支払に使えれば割増金を引き上げられたが、オスマン債務管理局は経費の1/4を債券回収に用いると決定した。
その後、5月2日のケンタッキーダ

  3%|▎         | 36/1070 [00:00<00:08, 119.03it/s]

これらの作品は、1995年にポーランド財務省が、対外債務管理会社との調停の末に取得した約4000点のグリーンコレクションの一部である。
本商品の製造・販売元である那智黒総本舗の本社・工場は東牟婁郡太地町に所在する。
その後のリオドセは、BOVESPAに上場し、さらにはニューヨーク証券取引所にも外国預託証券という形式で上場を達成した。
観測にはすばる望遠鏡が用いられ、2004年12月12日から2007年3月21日にかけての観測で得られたデータ中から発見された。
また、「あいち国際女性映画祭2002」にて上映された。
2006年から2008年まで所属していたFCガンジュ岩手では主将で、選手兼コーチを務めた。
1995年5月の大統領選挙でシラクにジョスパンが敗北し下野したが、日程を繰り上げて行なわれた1997年6月の国民議会総選挙では、選挙前の予想に反して第一党となり、共産党や緑の党などとともに左翼連合政権を発足させ、リオネル・ジョスパンが首相に就任した。
ダック・テストは帰納法の一種である。
囲碁・将棋とも、講座とプロ棋戦の解説が多く放送され、棋界の最新情報を紹介する番組もある。
2014年6月2日、VfBシュトゥットガルトへ2018年までの4年契約で完全移籍した。
物語はこの学兵部隊の1つに配置された、速水厚志少年の視点から描かれることになる。
1979年南アフリカグランプリは、1979年F1世界選手権の第3戦として、1979年3月3日にキャラミで開催された。
ムバンダカ空港は、コンゴ民主共和国赤道州ムバンダカにある空港。
症状の重さを示す重症などとは異なる概念である。
2010年代の若手にはタンスマン国際作曲コンクールを制したオンジェイ・アダメクと、チェコ国際現代音楽祭のディレクターを務めたペトラ・バクラが輩出されており、前者はアルディッティ弦楽四重奏団が取り上げるなど、活動範囲を大きく広げる逸材として期待されている。
中古車は1991年に神奈川中央交通の車両を購入したのが最初だったが、譲受車の導入はJRバスの中でも最も早かった。
深澳漁港は、台湾新北市瑞芳区に存在する漁港である。
第二次世界大戦でドイツ軍がオランダとベルギーで初めて使い、ノルマンディー上陸作戦で連合国軍が多用した。
また、マヌエル・ノイアー、アリエン・ロッベンを抑えて2014年のUEFA欧州最優秀

  6%|▌         | 64/1070 [00:00<00:07, 126.85it/s]

...

1792年に、父親の事業のパートナーであったヤン・ラディスラフ・ドゥシークと結婚した。
フロイド・ランディスの、ツール・ド・フランスにおけるドーピング騒動の余波を受けて、2006年シーズン限りでフォナックが解散したため、2007年に、現在も在籍する、AG2R・ラ・モンディアルに移籍。
バタフライキーボードの正式な名称はTrackWriteだが、開発コードネームの「バタフライ」から、バタフライキーボードという名称が広がった。
一方で新渡戸稲造は在米中の1932年8月20日、CBSラジオでスティムソンドクトリンに反論する形で「満州事変と不戦条約」について言明しており、「満州事変は自己防衛の手段としてなされたものであって侵略ではなく、満州国は一般に考えられているように日本の傀儡政権ではない」と表明している。
それに従ってこの連盟も軟式野球連盟として始まり、準硬式野球連盟と改称したという経緯がある。
中川智正が持ってきた本に載っていたイラクのサリンプラントと似ていたことで、滝沢は設計に自信を持った。
上流部にはしのがやと公園、下流には水車公園や緑道には小川の流れる公園、桜並木が整備されている。
2019年に入ってから、香港民主化デモの影響で観光客が激減し、従業員の半数を削減し営業時間を短縮するなどの経営改善策を実施するものの、2020年には新型コロナウイルスの大流行で改善の見通しが立たず、2020年3月3日より一時閉店を決定した。
影響をうけたアーティストとアルバムは、シンディ・ローパーと彼女のアルバム「Twelve Deadly Cyns... and Then Some」。
1990年12月、第2次海部改造内閣で科学技術庁長官に任命され、史上6人目の女性閣僚として初入閣。
丸の内パークビルディング地下1階から4階、低層棟、三菱一号館、一号館広場からなる商業ゾーンは総称として丸の内ブリックスクエアと名付けられている。
1996年、ロードレース界の名将、シリル・ギマールがチーム・マネジャーとなり結成。
1895年の憲法制定会議のとき、別の者が提案した「一滴ルール」を州が採用することを支持した。
1992年10月5日、講談社より発売された。
2015年、出身地の埼玉県に新たに発足した武蔵ヒートベアーズから、分配ドラフトにおいて「地元枠移籍」で指名を受けて移籍した。
国民の「夢

 98%|█████████▊| 1044/1070 [00:07<00:00, 130.17it/s]

直線に入っても上がり3ハロン34秒4のメンバー中3位の伸びを見せ、脚色が衰えずに後続に2馬身突き放して、3勝目を挙げた。
1997年株式会社化し、社名を株式会社四季社と改める。
負傷した李鴻章に代わって李経方参議が欽差全権大臣となった。
ボールドウィン社はギブソン・ギター・コーポレーションによって買われた。
番組ご意見番出川哲朗が出川ガールを連れて、不器用ながらリアクション芸人魂を燃やし体を張って挑戦していく企画。
11月の講道館杯では決勝まで進むと、予てより目標にしてきた高校の4年先輩で元世界2位である了徳寺学園職員の緒方亜香里を指導2で破って初優勝を果たした。
松友美佐紀は、日本のバドミントン選手。
高校卒業年となる2018年には単独ライブを開催し、目標であったAnimelo Summer Liveに初出演する。
1823年にチヌーク族の部族長の娘であるプリンセス・レーヴァンと結婚する。
尖沙咀の突端にあり、ハーバービューの客室とロビーなどからビクトリア・ハーバーと香港島の景色を目の前に見ることができ「香港のホテル最高の眺望」と称される眺望の良さを誇る上、当時として最新の設備を誇ったため、オープン当時からザ・ペニンシュラ香港やマンダリン・オリエンタル香港、カオルーン・シャングリ・ラと並ぶ香港を代表する高級ホテルとして君臨することとなった。
MAXのメンバー。
2017年2月27日に北京国安へ移籍。
大連市の星海広場にも華表が1本あり、天安門の前の物より大きいと土地っ子の自慢であったが、天安門のものより大きいことと、不祥事で失脚した薄煕来が大連市長時代に建設したことが原因で2016年8月に取り壊された。
同宗派の真龍寺・宝林寺とともに「三軒寺」を称している。
海上自衛隊のUS-2や航空自衛隊のC-1が利用されることもある。
その時点ではビリー・バークがモンロー将軍役であった。
2018年からクリシューマECで主力メンバーとして活動したが、2020年1月18日、日本のレノファ山口FCへの加入が発表された。
堺市立浜寺中学校は、大阪府堺市西区にある公立中学校。
特にハモンドオルガンと接続して用いられることが多いがハモンド社の製品ではない。
トロポスフィアは、コンゴ民主共和国の企業Developpement Tous Azimuts社によって2007年から開発されている

100%|██████████| 1070/1070 [00:08<00:00, 131.21it/s]

あいちトリエンナーレが愛知県や名古屋市などからの税金、県や複数企業からの後援を受けたうえで開催されていたのに対し、こちらは全て個人個人からの資金により開催され、県や企業からの後援を一切受けていない。
2013年2月27日から発売。
当日は、マリオやJ.L.ボニエ、宮本茂、マーク・ウッドベリーも登壇した。
富士見書房の「マギウス」のサプリメントとしてテーブルトークRPG化もされている。
2012年、ロンドンオリンピックの代表12名に選ばれ、本大会に出場した。
会津バスと共通の、白い車体に赤と青のラインを描いたボディが特徴で、そのほとんどがコンフォートだが、黒塗りのセドリックも存在する。
2019年、宗像サニックスブルースに加入。
リバティー制度が守られず外泊許可を出していた件に関し、事件後の4月24日、米軍キャンプ瑞慶覧で、事件に抗議した北谷町の町議員団が米海兵隊政務外交部長のダリン・クラーク大佐と面談した際に、「99.9%の兵士が制度を順守している」「逆に、なぜリバティー制度が不安を与えているのか聞きたい」と逆ぎれともとれる発言があった。
その他プエブロを訪れた国の指導者としては、ジョン・F・ケネディ大統領、ビル・クリントン大統領、ジョン・ケリー上院議員およびアル・ゴア副大統領がいた。
1954年のリーグ初優勝から西鉄ライオンズの黄金時代が始まり、豊田泰光、中西太、大下弘、稲尾和久らを擁したチームは1956~1958年に読売ジャイアンツを下し日本シリーズ3連覇を果たしたが、特に1958年は3連敗の後の雨天中止が引き金となり、奇跡とも言われた4連勝を果たした。
しかし彼は1878年に小惑星を1つ発見している。
1988年にオーストラリア航空がイーストウェスト航空から株式の25%を買収し1991年に完全な子会社になったが、その後オーストラリア航空がカンタス航空に買収されたことにより1992年にカンタス航空グループの一員となった。
寛永13年には細川忠利によって水前寺が建立され、これは木山方向への街道に沿った茶屋として発達し、細川綱利時代には桃山文化風の回遊式庭園・成趣園が完成した。
中部日本教映株式会社は、愛知県一宮市にある教育映画などの製作・配給を行う会社である。
2019年現在、ジェイテクトが筆頭株主で、主要株主に日野自動車も名を連ねている。
1923年チリのサンテ

In [18]:
# 8-18
print("# 正解")
print(entities_list[0])
print("# 抽出")
print(entities_predicted_list[0])

# 正解
[{'name': 'IAU100 Name ExoWorldsプロジェクト', 'span': [41, 68], 'type_id': 7}, {'name': 'オーストリア共和国', 'span': [93, 102], 'type_id': 5}]
# 抽出
[{'name': 'IAU100 Na', 'span': [41, 50], 'type_id': 8}, {'name': 'me ExoWorldsプロジェクト', 'span': [50, 68], 'type_id': 7}, {'name': 'HAT-P-14', 'span': [74, 82], 'type_id': 7}, {'name': 'HAT-P-14', 'span': [83, 91], 'type_id': 7}, {'name': 'オーストリア共和国', 'span': [93, 102], 'type_id': 5}]


In [19]:
# 8-19
def evaluate_model(entities_list, entities_predicted_list, type_id=None):
    """
    正解と予測を比較し、モデルの固有表現抽出の性能を評価する。
    type_idがNoneのときは、全ての固有表現のタイプに対して評価する。
    type_idが整数を指定すると、その固有表現のタイプのIDに対して評価を行う。
    """
    num_entities = 0 # 固有表現(正解)の個数
    num_predictions = 0 # BERTにより予測された固有表現の個数
    num_correct = 0 # BERTにより予測のうち正解であった固有表現の数

    # それぞれの文章で予測と正解を比較。
    # 予測は文章中の位置とタイプIDが一致すれば正解とみなす。
    for entities, entities_predicted \
        in zip(entities_list, entities_predicted_list):

        if type_id:
            entities = [ e for e in entities if e['type_id'] == type_id ]
            entities_predicted = [ 
                e for e in entities_predicted if e['type_id'] == type_id
            ]
            
        get_span_type = lambda e: (e['span'][0], e['span'][1], e['type_id'])
        set_entities = set( get_span_type(e) for e in entities )
        set_entities_predicted = \
            set( get_span_type(e) for e in entities_predicted )

        num_entities += len(entities)
        num_predictions += len(entities_predicted)
        num_correct += len( set_entities & set_entities_predicted )

    # 指標を計算
    precision = num_correct/num_predictions # 適合率
    recall = num_correct/num_entities # 再現率
    f_value = 2*precision*recall/(precision+recall) # F値

    result = {
        'num_entities': num_entities,
        'num_predictions': num_predictions,
        'num_correct': num_correct,
        'precision': precision,
        'recall': recall,
        'f_value': f_value
    }

    return result

In [20]:
# 8-20
print( evaluate_model(entities_list, entities_predicted_list) )

{'num_entities': 2607, 'num_predictions': 2763, 'num_correct': 2309, 'precision': 0.8356858487151647, 'recall': 0.8856923667050249, 'f_value': 0.8599627560521416}


In [21]:
# 8-21
class NER_tokenizer_BIO(BertJapaneseTokenizer):

    # 初期化時に固有表現のカテゴリーの数`num_entity_type`を
    # 受け入れるようにする。
    def __init__(self, *args, **kwargs):
        self.num_entity_type = kwargs.pop('num_entity_type')
        super().__init__(*args, **kwargs)

    def encode_plus_tagged(self, text, entities, max_length):
        """
        文章とそれに含まれる固有表現が与えられた時に、
        符号化とラベル列の作成を行う。
        """
        # 固有表現の前後でtextを分割し、それぞれのラベルをつけておく。
        splitted = [] # 分割後の文字列を追加していく
        position = 0
        for entity in entities:
            start = entity['span'][0]
            end = entity['span'][1]
            label = entity['type_id']
            splitted.append({'text':text[position:start], 'label':0})
            splitted.append({'text':text[start:end], 'label':label})
            position = end
        splitted.append({'text': text[position:], 'label':0})
        splitted = [ s for s in splitted if s['text'] ]

        # 分割されたそれぞれの文章をトークン化し、ラベルをつける。
        tokens = [] # トークンを追加していく
        labels = [] # ラベルを追加していく
        for s in splitted:
            tokens_splitted = self.tokenize(s['text'])
            label = s['label']
            if label > 0: # 固有表現
                # まずトークン全てにI-タグを付与
                labels_splitted =  \
                    [ label + self.num_entity_type ] * len(tokens_splitted)
                # 先頭のトークンをB-タグにする
                labels_splitted[0] = label
            else: # それ以外
                labels_splitted =  [0] * len(tokens_splitted)
            
            tokens.extend(tokens_splitted)
            labels.extend(labels_splitted)

        # 符号化を行いBERTに入力できる形式にする。
        input_ids = self.convert_tokens_to_ids(tokens)
        encoding = self.prepare_for_model(
            input_ids, 
            max_length=max_length, 
            padding='max_length',
            truncation=True
        ) 

        # ラベルに特殊トークンを追加
        labels = [0] + labels[:max_length-2] + [0]
        labels = labels + [0]*( max_length - len(labels) )
        encoding['labels'] = labels

        return encoding

    def encode_plus_untagged(
        self, text, max_length=None, return_tensors=None
    ):
        """
        文章をトークン化し、それぞれのトークンの文章中の位置も特定しておく。
        IO法のトークナイザのencode_plus_untaggedと同じ
        """
        # 文章のトークン化を行い、
        # それぞれのトークンと文章中の文字列を対応づける。
        tokens = [] # トークンを追加していく。
        tokens_original = [] # トークンに対応する文章中の文字列を追加していく。
        words = self.word_tokenizer.tokenize(text) # MeCabで単語に分割
        for word in words:
            # 単語をサブワードに分割
            tokens_word = self.subword_tokenizer.tokenize(word) 
            tokens.extend(tokens_word)
            if tokens_word[0] == '[UNK]': # 未知語への対応
                tokens_original.append(word)
            else:
                tokens_original.extend([
                    token.replace('##','') for token in tokens_word
                ])

        # 各トークンの文章中での位置を調べる。（空白の位置を考慮する）
        position = 0
        spans = [] # トークンの位置を追加していく。
        for token in tokens_original:
            l = len(token)
            while 1:
                if token != text[position:position+l]:
                    position += 1
                else:
                    spans.append([position, position+l])
                    position += l
                    break

        # 符号化を行いBERTに入力できる形式にする。
        input_ids = self.convert_tokens_to_ids(tokens) 
        encoding = self.prepare_for_model(
            input_ids, 
            max_length=max_length, 
            padding='max_length' if max_length else False, 
            truncation=True if max_length else False
        )
        sequence_length = len(encoding['input_ids'])
        # 特殊トークン[CLS]に対するダミーのspanを追加。
        spans = [[-1, -1]] + spans[:sequence_length-2] 
        # 特殊トークン[SEP]、[PAD]に対するダミーのspanを追加。
        spans = spans + [[-1, -1]] * ( sequence_length - len(spans) ) 

        # 必要に応じてtorch.Tensorにする。
        if return_tensors == 'pt':
            encoding = { k: torch.tensor([v]) for k, v in encoding.items() }

        return encoding, spans

    @staticmethod
    def Viterbi(scores_bert, num_entity_type, penalty=10000):
        """
        Viterbiアルゴリズムで最適解を求める。
        """
        m = 2*num_entity_type + 1
        penalty_matrix = np.zeros([m, m])
        for i in range(m):
            for j in range(1+num_entity_type, m):
                if not ( (i == j) or (i+num_entity_type == j) ): 
                    penalty_matrix[i,j] = penalty
        
        path = [ [i] for i in range(m) ]
        scores_path = scores_bert[0] - penalty_matrix[0,:]
        scores_bert = scores_bert[1:]

        for scores in scores_bert:
            assert len(scores) == 2*num_entity_type + 1
            score_matrix = np.array(scores_path).reshape(-1,1) \
                + np.array(scores).reshape(1,-1) \
                - penalty_matrix
            scores_path = score_matrix.max(axis=0)
            argmax = score_matrix.argmax(axis=0)
            path_new = []
            for i, idx in enumerate(argmax):
                path_new.append( path[idx] + [i] )
            path = path_new

        labels_optimal = path[np.argmax(scores_path)]
        return labels_optimal

    def convert_bert_output_to_entities(self, text, scores, spans):
        """
        文章、分類スコア、各トークンの位置から固有表現を得る。
        分類スコアはサイズが（系列長、ラベル数）の2次元配列
        """
        assert len(spans) == len(scores)
        num_entity_type = self.num_entity_type
        
        # 特殊トークンに対応する部分を取り除く
        scores = [score for score, span in zip(scores, spans) if span[0]!=-1]
        spans = [span for span in spans if span[0]!=-1]

        # Viterbiアルゴリズムでラベルの予測値を決める。
        labels = self.Viterbi(scores, num_entity_type)

        # 同じラベルが連続するトークンをまとめて、固有表現を抽出する。
        entities = []
        for label, group \
            in itertools.groupby(enumerate(labels), key=lambda x: x[1]):
            
            group = list(group)
            start = spans[group[0][0]][0]
            end = spans[group[-1][0]][1]

            if label != 0: # 固有表現であれば
                if 1 <= label <= num_entity_type:
                     # ラベルが`B-`ならば、新しいentityを追加
                    entity = {
                        "name": text[start:end],
                        "span": [start, end],
                        "type_id": label
                    }
                    entities.append(entity)
                else:
                    # ラベルが`I-`ならば、直近のentityを更新
                    entity['span'][1] = end 
                    entity['name'] = text[entity['span'][0]:entity['span'][1]]
                
        return entities

In [22]:
# 8-22
# トークナイザのロード
# 固有表現のカテゴリーの数`num_entity_type`を入力に入れる必要がある。
tokenizer = NER_tokenizer_BIO.from_pretrained(
    MODEL_NAME,
    num_entity_type=8 
)

# データセットの作成
max_length = 128
dataset_train_for_loader = create_dataset(
    tokenizer, dataset_train, max_length
)
dataset_val_for_loader = create_dataset(
    tokenizer, dataset_val, max_length
)

# データローダの作成
dataloader_train = DataLoader(
    dataset_train_for_loader, batch_size=32, shuffle=True
)
dataloader_val = DataLoader(dataset_val_for_loader, batch_size=256)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'NER_tokenizer_BIO'.


In [23]:
# 8-23

# ファインチューニング
checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='model_BIO/'
)

trainer = pl.Trainer(
    gpus=1,
    max_epochs=5,
    callbacks=[checkpoint]
)

# PyTorch Lightningのモデルのロード
num_entity_type = 8
num_labels = 2*num_entity_type+1
model = BertForTokenClassification_pl(
    MODEL_NAME, num_labels=num_labels, lr=1e-5
)

# ファインチューニング
trainer.fit(model, dataloader_train, dataloader_val)
best_model_path = checkpoint.best_model_path

# 性能評価
model = BertForTokenClassification_pl.load_from_checkpoint(
    best_model_path
) 
bert_tc = model.bert_tc.cuda()

entities_list = [] # 正解の固有表現を追加していく
entities_predicted_list = [] # 抽出された固有表現を追加していく
for sample in tqdm(dataset_test):
    text = sample['text']
    encoding, spans = tokenizer.encode_plus_untagged(
        text, return_tensors='pt'
    )
    encoding = { k: v.cuda() for k, v in encoding.items() } 
    
    with torch.no_grad():
        output = bert_tc(**encoding)
        scores = output.logits
        scores = scores[0].cpu().numpy().tolist()
        
    # 分類スコアを固有表現に変換する
    entities_predicted = tokenizer.convert_bert_output_to_entities(
        text, scores, spans
    )

    entities_list.append(sample['entities'])
    entities_predicted_list.append( entities_predicted )

print(evaluate_model(entities_list, entities_predicted_list))

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSeq

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the m

{'num_entities': 2607, 'num_predictions': 2686, 'num_correct': 2301, 'precision': 0.8566641846612063, 'recall': 0.8826237054085155, 'f_value': 0.8694502172680899}
